## 并发编程

不管在Python或者Java甚至js中都需要并发编程的存在.并发编程的目的是为了尽可能的使用机器的资源, 以达到更高的单机性能, 进而提升性价比.

这里有2个概念.
 * `并发`: 指同时处理多个事情的能力
 * `并行`: 指同一个时间处理多个事情

常用的并发编程有几种形式:
 * `多线程`: 通过将不同的工作分配到不同线程的方式
 * `多进程`: 通过将不同的工作分配到不同进程的方式
 * `协程`: 自定义实现的调度器, 每个任务在需要等待的时候主动交出控制权到其他任务中

### 线程与进程

* 进程是计算机中的程序关于某数据集合上的一次运行活动，是 __系统__ 进行资源分配和调度的基本单位.
* 线程是操作系统能够进行运算调度的最小单位。它被包含在进程之中，是进程中的实际运作单位。

简而言之: 操作系统管理进程, 进程可以创建或者销毁线程(主线程除外), 但是线程由系统进行调度.

### 多线程

在使用多线程过程中, 一般会使用Master/Worker形式. 我们先从最简单的多线程开始, 再到到通用的多线程模型.

在Python中使用多线程需要引入`threading`库, 使用其中的`Thread`类来创建线程.

In [ ]:
import time
# 引入多线程库
import threading

def say_hello(name):
    for i in range(10):
        print("hello {}".format(name))

thread1 = threading.Thread(target=say_hello, args=('small red',))
thread2 = threading.Thread(target=say_hello, args=('small light',))

thread1.start()
thread2.start()

在以上代码中, 我们创建了2个线程分别根据参数进行打印操作. 思考为什么一个打印完了之后才打印另一个?

#### Master/Worker形式

在通常开发中, 一般很少直接将某一部分业务直接分配给一个方法. 更通用的做法是主线程进行分发任务, worker根据master给出的任务执行对应的工作.

In [ ]:
def worker(thread_id: int, queue: Queue):
    while True:
        data = queue.get(True)
        print("worker-{} receive task: ".format(thread_id), data)
        time.sleep(data)


if __name__ == '__main__':
    threads = []
    task_queue = Queue()
    for i in range(3):
        threads.append(Thread(target=worker, args=(i, task_queue), daemon=True))
    list(map(lambda t: t.start(), threads))

    for i in range(10):
        task_queue.put(random.randint(1, 3))
    time.sleep(10)

在以上例子中, 我们通过在主线程中往队列中发送"任务"来达到分配任务的目的.
而worker中通过从队列中获取数据, 并根据任务数据执行相应的任务.

在这个过程中, worker只要一做完当前工作, 就会在队列处等待新任务的到来.
这样就可以确保任务能被第一时间消化, 工作非常的饱和. 自然性价比就出来了.

#### 多线程中的竞态问题

在多线程中, 如果多个线程同时对一个资源进行操作, 那后果将是灾难性的. 思考以下例子, 请思考输出和输出该值的原因的是什么

In [ ]:
amount = 0


def worker(count):
    global amount
    for i in range(count):
        amount = amount + 1


if __name__ == '__main__':
    t1 = Thread(target=worker, args=(10,))
    t2 = Thread(target=worker, args=(20,))
    t3 = Thread(target=worker, args=(30,))

    t1.start()
    t2.start()
    t3.start()
    t3.join()

    print(amount)

#### 同步锁机制

解决以上问题的方法就是增加一个锁, 在修改时防止其他线程修改. 确保同一时刻只有一个线程在操作共享变量.

In [ ]:
amount = 0
lock = Lock()


def worker(count):
    global amount
    for i in range(count):
        lock.acquire(True)
        amount = amount + 1
        lock.release()

请思考以上还有可能出现什么问题?正确的做法是什么?

## 多进程

多进程与多线程基本类似, 不同的是, 多进程是以自身为蓝本创建一个新的进程, 并且开辟一片新的内存空间.需要注意的点有以下几点.
 * 多进程会采用写时复制技术来降低创建进程带来的内存拷贝开销, 共享变量只有在修改时才会进行拷贝
 * 多进程的开销会比多线程来的更大, 但是可以实现更好的并发效果
 * 多进程编程中不需要考虑变量之间的同步关系, 但是还是需要注意进程之外的同步问题(文件读写)
 * 多进程中一般使用临界区, 互斥量, 信号量或者事件来进行同步操作
 
python中使用`multiprocessing`库来支持多进程编程

In [ ]:
import time
# 引入多进程支持
from multiprocessing import Process


def say_hello(name):
    for i in range(10):
        print("hello {}".format(name))
        time.sleep(1)


if __name__ == '__main__':
    process1 = Process(target=say_hello, args=('small red',))
    process2 = Process(target=say_hello, args=('small light',))

    process1.start()
    process2.start()

#### 写时复制

写入时复制是一种计算机程序设计领域的优化策略。其核心思想是，如果有多个调用者同时请求相同资源（如内存或磁盘上的数据存储），他们会共同获取相同的指针指向相同的资源，直到某个调用者试图修改资源的内容时，系统才会真正复制一份专用副本（private copy）给该调用者，而其他调用者所见到的最初的资源仍然保持不变。这个过程对其他的调用者是透明的. 我们来验证一下

In [1]:
var = 0


def worker(worker_id):
    global var
    print(worker_id, id(var), var)
    var = worker_id
    print(worker_id, id(var), var)


if __name__ == '__main__':
    process1 = Process(target=worker, args=(1,))
    process2 = Process(target=worker, args=(2,))

    print(id(var), var)
    process1.start()
    process2.start()
    process2.join()
    print(id(var), var)

NameError: name 'Process' is not defined

从以上程序的输出, 我们可以看到, 操作系统在我们修改变量的时候才会拷贝这个变量.

同时需要注意到, 我们针对全局变量的修改会被全部隔离开.

#### 多进程中的锁

多进程中的加锁方式与多线程中加锁方式, 在代码上并无区别, 但是在实现原理上却并不一样. 思考下多进程加锁如何实现?

In [ ]:
lock = Lock()


def worker(count):
    for i in range(count):
        # lock.acquire(True)
        with open('amount.txt', 'r+') as w:
            amount = str(int(w.read()) + 1)
            w.seek(0)
            w.write(amount)
        # lock.release()


if __name__ == '__main__':
    with open('amount.txt', 'w') as fp:
        fp.write('0')

    p1 = Process(target=worker, args=(1000,))
    p2 = Process(target=worker, args=(2000,))
    p3 = Process(target=worker, args=(3000,))

    p1.start()
    p2.start()
    p3.start()
    p3.join()

## 协程

协程与多线程多进程的原理完全不用, 协程使用过在一个线程内尽可能的执行更多的指令以达到并发处理的能力.相对比与多线程与多线程
 * 协程更加轻量化, 一个协程就是一个函数, 在协程之间切换只需要切换栈空间和寄存器内容即可.
 * 协程只适用于IO密集型任务, 不适用于计算密集型任务.
 * 协程能最大化利用单核性能, 但是并不能最大化利用多核性能.(新的方向是同时使用多进程和协程)

python中为协程提供支持的是`asyncio`这个库. 我们这边只做简单介绍.

In [ ]:
import asyncio, threading


async def hello():
    print('Hello World! [THREAD %s] 1' % (threading.currentThread()))
    await asyncio.sleep(2)
    print('Hello Python! [THREAD %s] 1' % (threading.currentThread()))


async def world():
    print('Hello World! [THREAD %s] 2' % (threading.currentThread()))
    await asyncio.sleep(5)
    print('Hello Python! [THREAD %s] 2' % (threading.currentThread()))


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.wait([hello(), world()]))
    loop.close()

以下使用`httpx`库配合使用`asyncio`来实现快速的抓取网页

In [ ]:
import asyncio
import httpx


async def main():
    client = httpx.AsyncClient()
    for i in range(1000):
        print(await client.get('http://www.baidu.com'))


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    loop.run_until_complete(main())
    loop.close()

### asyncio原理

Python的`asyncio`实现的原理就是对`generator`的极致应用.
 * 使用yield交出控制权
 * 使用调度器调度所有的协程(生成器)
 
以下我们实现个简单的自己的asyncio

In [ ]:
# 查看sources目录内容